In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data", one_hot=True)


C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-pack

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Parameters

# Neural Network Parameters
n_hidden_1 = 392
n_hidden_2 = 256
n_feature = 784
n_classes = 10


In [3]:
X = tf.placeholder("float", [None, n_feature], name='image_features')
Y = tf.placeholder('float', [None, n_classes], name='labels')
print("X :", X)
print("Y :", Y)

X : Tensor("image_features:0", shape=(?, 784), dtype=float32)
Y : Tensor("labels:0", shape=(?, 10), dtype=float32)


In [4]:
weights = {'w1': tf.Variable(tf.random_normal([n_feature, n_hidden_1])),
           'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
           'w_out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))}
biases = {'b1': tf.Variable(tf.random_normal([n_hidden_1])),
          'b2': tf.Variable(tf.random_normal([n_hidden_2])),
          'b_out': tf.Variable(tf.random_normal([n_classes]))}


Instructions for updating:
Colocations handled automatically by placer.


In [5]:
print("weights   =>   ", weights)
print("biases     =>   ", biases)

weights   =>    {'w1': <tf.Variable 'Variable:0' shape=(784, 392) dtype=float32_ref>, 'w2': <tf.Variable 'Variable_1:0' shape=(392, 256) dtype=float32_ref>, 'w_out': <tf.Variable 'Variable_2:0' shape=(256, 10) dtype=float32_ref>}
biases     =>    {'b1': <tf.Variable 'Variable_3:0' shape=(392,) dtype=float32_ref>, 'b2': <tf.Variable 'Variable_4:0' shape=(256,) dtype=float32_ref>, 'b_out': <tf.Variable 'Variable_5:0' shape=(10,) dtype=float32_ref>}


In [6]:
# forward
def neural_network(x):
    layer1 = tf.add(tf.matmul(x,weights['w1']),biases['b1']) # layer 1 = 1x392
    layer2 = tf.add(tf.matmul(layer1,weights['w2']),biases['b2']) # Layer 2 = 1x256
    out_layer = tf.add(tf.matmul(layer2,weights['w_out']),biases['b_out']) # out_layer = 1x10
    return out_layer

logits = neural_network(X) # X = 15*784 -> logit = 15*10 -> 

# Compute loss and optimizer
loss_op =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels= Y))
# backwark
optimizer = tf.train.AdamOptimizer(learning_rate=0.1)  
train_op = optimizer.minimize(loss_op)

# Evaluate models 
correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
init = tf.global_variables_initializer()
training_epochs = 100

tf.summary.scalar("loss_show", loss_op)
tf.summary.scalar("accuracy_show", accuracy)
merged_summary_op = tf.summary.merge_all()
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logdir='./logdir2', graph=tf.get_default_graph())
    for epoch in range(training_epochs):
        # 1 iterator
        #print(sess.run(loss_op))
        image_batch, label_batch = mnist.train.next_batch(10000)
        sess.run(train_op, feed_dict={X: image_batch, Y: label_batch})
#         sess.run(train_op, feed_dict={X: mnist.train.images, Y: mnist.train.labels})
        #Display logs per epoch step
        #if (epoch+1) % 10 == 0:
        loss, summary = sess.run([loss_op, merged_summary_op], feed_dict={X: image_batch, Y: label_batch})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(loss))
        summary_writer.add_summary(summary,epoch)
    print("Optimization Finished!")
    print ("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    

summary_writer.close()

Epoch: 0001 cost= 22356.611328125
Epoch: 0002 cost= 17942.232421875
Epoch: 0003 cost= 20631.224609375
Epoch: 0004 cost= 25383.718750000
Epoch: 0005 cost= 14429.401367188
Epoch: 0006 cost= 6962.776855469
Epoch: 0007 cost= 8656.952148438
Epoch: 0008 cost= 7682.372070312
Epoch: 0009 cost= 5840.162597656
Epoch: 0010 cost= 5555.145019531
Epoch: 0011 cost= 4342.921386719
Epoch: 0012 cost= 3691.211669922
Epoch: 0013 cost= 4007.921630859
Epoch: 0014 cost= 3702.884765625
Epoch: 0015 cost= 3496.976806641
Epoch: 0016 cost= 3285.340820312
Epoch: 0017 cost= 3016.093994141
Epoch: 0018 cost= 2827.676757812
Epoch: 0019 cost= 2749.704589844
Epoch: 0020 cost= 2478.308105469
Epoch: 0021 cost= 2471.728515625
Epoch: 0022 cost= 2212.825683594
Epoch: 0023 cost= 2125.299560547
Epoch: 0024 cost= 2006.298828125
Epoch: 0025 cost= 1855.279418945
Epoch: 0026 cost= 1669.106201172
Epoch: 0027 cost= 1708.439208984
Epoch: 0028 cost= 1603.270507812
Epoch: 0029 cost= 1594.173095703
Epoch: 0030 cost= 1458.330078125
Epoch